In [ ]:
import os
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
data_folder = "data/"

In [ ]:
path_to_calib_data = os.path.join(os.getcwd(), 'data/training/calibration/calibration_data_synthetic.json')

In [ ]:
with open(path_to_calib_data, 'r') as F:
    data = json.load(F)

calib_data = {}
for key, value in data.items():
    calib_data[key] = np.array(value)

for name, matrix in calib_data.items():
    print(f"Matrix {name}:\n{matrix}\n")

In [ ]:
left_image_filename = "image_left.png"
right_image_filename = "image_right.png"

In [ ]:
img_left_color = cv2.imread(os.path.join(data_folder, left_image_filename))
img_right_color = cv2.imread(os.path.join(data_folder, right_image_filename))

In [ ]:
img_left_bw = cv2.blur(cv2.cvtColor(img_left_color, cv2.COLOR_RGB2GRAY),(5,5))
img_right_bw = cv2.blur(cv2.cvtColor(img_right_color, cv2.COLOR_RGB2GRAY),(5,5))

In [ ]:
def write_ply(fn, verts, colors):
    ply_header = '''ply
format ascii 1.0
element vertex %(vert_num)d
property float x
property float y
property float z
property uchar red
property uchar green
property uchar blue
end_header
'''
    out_colors = colors.copy()
    verts = verts.reshape(-1, 3)
    verts = np.hstack([verts, out_colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

In [ ]:
plt.imshow(img_left_bw, cmap='gray')
plt.show()

In [ ]:
plt.imshow(img_right_bw, cmap='gray')
plt.show()

In [ ]:
def compute_disparity(numDisparities, blockSize):
    stereo = cv2.StereoBM_create(numDisparities=numDisparities, blockSize=blockSize)
    disparity = stereo.compute(img_left_bw, img_right_bw)
    plt.imshow(disparity, cmap='gray')
    plt.colorbar()
    plt.show()

In [ ]:
numDisparities_slider = widgets.IntSlider(min=16, max=16 * 100, step=16, value=16 * 50, description='numDisparities:')
blockSize_slider = widgets.IntSlider(min=5, max=51, step=2, value=5, description='blockSize:')
interact(compute_disparity, numDisparities=numDisparities_slider, blockSize=blockSize_slider)
None

In [ ]:
stereo = cv2.StereoBM_create(numDisparities=16 * 50, blockSize=5) #numDisparities=96, blockSize=11)
disparity = stereo.compute(img_left_bw, img_right_bw)

In [ ]:
img = disparity.copy()
plt.imshow(img, 'CMRmap_r')
plt.show()

In [ ]:
# Calculate depth-to-disparity
R1, R2, P1, P2, Q, validPixROI1, validPixROI2 = cv2.stereoRectify(
    cameraMatrix1 = calib_data['CM'], distCoeffs1 = calib_data['dist'],
    cameraMatrix2 = calib_data['CM'], distCoeffs2 = calib_data['dist'],
    imageSize = img_left_color.shape[:2],
    R = calib_data['R'],
    T = calib_data['T'],
    flags=0, # cv.CALIB_ZERO_DISPARITY,
    alpha=-1
)

In [ ]:
cam1_map1, cam1_map2 = cv2.initUndistortRectifyMap(
    calib_data['CM'],
    calib_data['dist'],
    R1, P1, 
    img_left_color.shape[:2],
    cv2.CV_32FC1
)
cam2_map1, cam2_map2 = cv2.initUndistortRectifyMap(
    calib_data['CM'],
    calib_data['dist'], 
    R2, P2,
    img_left_color.shape[:2],
    cv2.CV_32FC1
)

In [ ]:
def draw_epipolar_lines(img1, img2, color=255, step=250):
    img1_lines = img1.copy()
    img2_lines = img2.copy()

    height = img1.shape[0]

    for y in range(0, height, step):
        cv2.line(img1_lines, (0, y), (img1.shape[1], y), color, 7)
        cv2.line(img2_lines, (0, y), (img2.shape[1], y), color, 7)

    return img1_lines, img2_lines

In [ ]:
rectified_cam1_img = cv2.remap(img_left_color, cam1_map1, cam1_map2, interpolation=cv2.INTER_LINEAR)
rectified_cam2_img = cv2.remap(img_right_color, cam2_map1, cam2_map2, interpolation=cv2.INTER_LINEAR)

epipolar_img_left, epipolar_img_right = draw_epipolar_lines(rectified_cam1_img, rectified_cam2_img)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(epipolar_img_left, cmap='gray')
plt.title('Rectified Left Camera Image with Epipolar Lines')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(epipolar_img_right, cmap='gray')
plt.title('Rectified Right Camera Image with Epipolar Lines')
plt.axis('off')

plt.show()

In [ ]:
def show_rotation(R):
    sy = np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    singular = sy < 1e-6
    if not singular:
        yaw = np.arctan2(R[2, 1], R[2, 2])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = np.arctan2(R[1, 0], R[0, 0])
    else:
        yaw = np.arctan2(-R[1, 2], R[1, 1])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = 0
    euler_angles_calculated = np.degrees(np.array([yaw, pitch, roll]))
    print(f"Yaw: {euler_angles_calculated[0]:.2f} degrees")
    print(f"Pitch: {euler_angles_calculated[1]:.2f} degrees")
    print(f"Roll: {euler_angles_calculated[2]:.2f} degrees")

In [ ]:
show_rotation(R1)
show_rotation(R2)

In [ ]:
points = cv2.reprojectImageTo3D(img, Q)

#reflect on x axis
reflect_matrix = np.identity(3)
reflect_matrix[0] *= -1
points = np.matmul(points, reflect_matrix)

#extract colors from image
colors = cv2.cvtColor(img_left_color, cv2.COLOR_BGR2RGB)

#filter by min disparity
mask = img > img.min()
out_points = points[mask]
out_colors = colors[mask]

#filter by dimension
idx = np.fabs(out_points[:,0]) < 4.5
out_points = out_points[idx]
out_colors = out_colors.reshape(-1, 3)
out_colors = out_colors[idx]

In [ ]:
write_ply('out.ply', out_points, out_colors)
print('%s saved' % 'out.ply')

In [ ]:
reflected_pts = np.matmul(out_points, reflect_matrix)
projected_img,_ = cv2.projectPoints(reflected_pts, np.identity(3), np.array([0., 0., 0.]), \
                          calib_data['CM'][:3, :3], np.array([0., 0., 0., 0.]))
projected_img = projected_img.reshape(-1, 2)
print(f"Min X: {projected_img[:, 0].min()}, Max X: {projected_img[:, 0].max()}")
print(f"Min Y: {projected_img[:, 1].min()}, Max Y: {projected_img[:, 1].max()}")
print(img_left_color.shape)

In [ ]:
def project_and_draw(rotation_angle_x, rotation_angle_y, rotation_angle_z):
    # Создание матрицы вращения по оси X
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(rotation_angle_x), -np.sin(rotation_angle_x)],
        [0, np.sin(rotation_angle_x), np.cos(rotation_angle_x)]
    ])
    
    # Создание матрицы вращения по оси Y
    Ry = np.array([
        [np.cos(rotation_angle_y), 0, np.sin(rotation_angle_y)],
        [0, 1, 0],
        [-np.sin(rotation_angle_y), 0, np.cos(rotation_angle_y)]
    ])
    
    Rz = np.array([
        [np.cos(rotation_angle_z), -np.sin(rotation_angle_z), 0],
        [np.sin(rotation_angle_z), np.cos(rotation_angle_z), 0],
        [0, 0, 1]
    ])
    
    # Общая матрица вращения
    R = np.dot(np.dot(Rx, Ry), Rz)

    # Применение матрицы вращения к точкам
    reflected_pts = np.dot(out_points, reflect_matrix)
    rotated_pts = np.dot(reflected_pts, R)
    
    # Выполнение проекции
    projected_img, _ = cv2.projectPoints(rotated_pts, np.eye(3), np.zeros(3), 
                                         calib_data['CM'][:3, :3],
                                         np.zeros(4))
    projected_img = projected_img.reshape(-1, 2)
    
    blank_img = np.zeros(img_left_color.shape, 'uint8')
    img_colors = img_right_color[mask][idx].reshape(-1, 3)
    
    for i, pt in enumerate(projected_img):
        pt_x = int(pt[0])
        pt_y = int(pt[1])
        if pt_x > 0 and pt_y > 0:
            col = (int(img_colors[i, 2]), int(img_colors[i, 1]), int(img_colors[i, 0]))
            cv2.circle(blank_img, (pt_x, pt_y), 1, col)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(blank_img)
    plt.axis('off')
    plt.show()


In [ ]:
from ipywidgets import interact, FloatSlider, FloatText, HBox, VBox
# Интерфейс для изменения параметров проекции
rotation_angle_x_slider = FloatSlider(value=0.0, min=-np.pi, max=np.pi, step=0.01, description='Rotate X:')
rotation_angle_x_text = FloatText(value=0.0, description='Rotate X:')
rotation_angle_y_slider = FloatSlider(value=0.0, min=-np.pi, max=np.pi, step=0.01, description='Rotate Y:')
rotation_angle_y_text = FloatText(value=0.0, description='Rotate Y:')
rotation_angle_z_slider = FloatSlider(value=0.0, min=-np.pi, max=np.pi, step=0.01, description='Rotate Z:')
rotation_angle_z_text = FloatText(value=0.0, description='Rotate Z:')

def sync_widgets(slider, text):
       def on_value_change(change):
           text.value = change['new']
       slider.observe(on_value_change, names='value')

       def on_text_change(change):
           slider.value = change['new']
       text.observe(on_text_change, names='value')

sync_widgets(rotation_angle_x_slider, rotation_angle_x_text)
sync_widgets(rotation_angle_y_slider, rotation_angle_y_text)
sync_widgets(rotation_angle_z_slider, rotation_angle_z_text)

ui = VBox([
    HBox([rotation_angle_x_slider, rotation_angle_x_text]),
    HBox([rotation_angle_y_slider, rotation_angle_y_text]),
    HBox([rotation_angle_z_slider, rotation_angle_z_text]),
])

# Вызов функции с виджетами
interact(project_and_draw,
         rotation_angle_x=rotation_angle_x_slider, 
         rotation_angle_y=rotation_angle_y_slider,
         rotation_angle_z=rotation_angle_z_slider)

In [ ]:
plt.imshow(img, 'CMRmap_r')
plt.show()

In [ ]:
%matplotlib widget

X = []
Y = []

def get_points_on_image(img, n_points=2):
    fig, ax = plt.subplots()
    ax.imshow(img)
    points = []

    out = widgets.Output()
    
    def onclick(event):
        if event.inaxes is not None:  # Проверка, что клик был внутри области изображения
            if len(points) < n_points:
                x, y = int(event.xdata), int(event.ydata)
                points.append((x, y))
                ax.plot(x, y, 'ro')  # Рисуем красные точки
                fig.canvas.draw()
                with out:
                    X.append(x)
                    Y.append(y)
                    print(f"Точка добавлена: ({x}, {y})")

            if len(points) == n_points:
                fig.canvas.mpl_disconnect(cid)
                plt.close()

    cid = fig.canvas.mpl_connect('button_press_event', onclick)
    display(out)
    plt.show()


# Захват точек
points = get_points_on_image(img, n_points=3)
print("Координаты точек:", points)

In [ ]:
print(X[0], Y[0])
print(X[1], Y[1])
print(X[2], Y[2])

In [ ]:
def reproject_to_3d(Q, x, y, disparity):
    point_2d = np.array([x, y, disparity[x][y], 1.0])
    point_3d = np.dot(Q, point_2d)
    point_3d = point_3d / point_3d[3]
    return point_3d[:3]

def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

x1, y1 = X[0], Y[0]
x2, y2 = X[1], Y[1]
x3, y3 = X[2], Y[2]

# Восстановление 3D точки
restored_3d_point_1 = reproject_to_3d(Q, x1, y1, disparity)
restored_3d_point_2 = reproject_to_3d(Q, x2, y2, disparity)
restored_3d_point_3 = reproject_to_3d(Q, x3, y3, disparity)
print(f"3D координаты точки 1: {restored_3d_point_1}")
print(f"3D координаты точки 2: {restored_3d_point_2}")
print(f"3D координаты точки 3: {restored_3d_point_3}")
print(f"Расстояние между точками 1 и 2: {euclidean_distance(restored_3d_point_1, restored_3d_point_2)}")
print(f"Расстояние между точками 2 и 3: {euclidean_distance(restored_3d_point_2, restored_3d_point_3)}")
print(f"Расстояние между точками 3 и 1: {euclidean_distance(restored_3d_point_3, restored_3d_point_1)}")